In [1]:
import cv2
import time
import os
import numpy as np
import natsort
import matplotlib.pyplot as plt
from pathlib import Path
import glob
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

# Input : Image Path

In [23]:
# Input_1: Query 파노라마 이미지
# Input_2: DB 파노라마 이미지
# Input_3: Query 파노라마 이미지 간판 검출 결과(class, x,y,w,h)
# Input_4: DB 파노라마 이미지 간판 검출 결과(class, x,y,w,h)
# (선택) Input_5: 파노라마 ID

# q_panorama = img
# db_panorama = img
# q_coord_list = [(class, x,y,w,h), ..., (class, x,y,w,h)]
# db_coord_list = [(class, x,y,w,h), ..., (class, x,y,w,h)]

match_score_dir = '/home/Signboard_Retrieval/roadview_384_crop/match_score/'
topk = 1

def crop_get(q_panorama, db_panorama, q_coord_list, db_coord_list, panorama_id=0):
    
    '''
    panorama_id : ID of panorama, specific location identified number including query and database
    '''
    
    #get query cropped image
    q_crop_list = [] #많아봤자 10개 이미지, 작은 크기이므로
    for q_coord in q_coord_list:
        
        # get coord
        _, float_x_center, float_y_center, float_width, float_height = q_coord
        del q_coord
        
        # crop image
        min_x = int(float_x_center - float_width / 2)
        min_y = int(float_y_center - float_height / 2)
        max_x = min_x + int(float_width)
        max_y = min_y + int(float_height)
        del float_x_center, float_y_center, float_width, float_height
        
        #cv2 crop
        croppedImage = q_panorama[min_y: max_y, min_x: max_x]

        # PIL Image crop
        # det_box = (min_x, min_y, max_x, max_y)
        # croppedImage = img_org.crop(det_box)
        
        #get cropped Image
        q_crop_list.append(croppedImage)
        del croppedImage
        
    #get db cropped image
    db_crop_list = [] #많아봤자 10개 이미지, 작은 크기이므로
    for db_coord in db_coord_list:
        
        # get coord
        _, float_x_center, float_y_center, float_width, float_height = db_coord
        del db_coord
        
        # crop image
        min_x = int(float_x_center - float_width / 2)
        min_y = int(float_y_center - float_height / 2)
        max_x = min_x + int(float_width)
        max_y = min_y + int(float_height)
        del float_x_center, float_y_center, float_width, float_height
        
        #cv2 crop
        croppedImage = db_panorama[min_y: max_y, min_x: max_x]

        # PIL Image crop
        # det_box = (min_x, min_y, max_x, max_y)
        # croppedImage = img_org.crop(det_box)
        
        #get cropped Image
        db_crop_list.append(croppedImage)
        del croppedImage
        
    return q_crop_list, db_crop_list, panorama_id

In [49]:
#### exec ####

with open('/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/test/db_coord_list.pkl', 'rb') as fp: #len은 동일
    db_coord_list = pickle.load(fp)
with open('/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/test/q_coord_list.pkl', 'rb') as fp: #len은 동일
    q_coord_list = pickle.load(fp)
with open('/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/test/db_panorama_list.pkl', 'rb') as fp: #len은 동일
    db_panorama_list = pickle.load(fp)
with open('/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/test/q_panorama_list.pkl', 'rb') as fp: #len은 동일
    q_panorama_list = pickle.load(fp)

q_panorama = q_panorama_list[0]
db_panorama = db_panorama_list[0]

q_crop_list, db_crop_list, panorama_id = crop_get(q_panorama, db_panorama, q_coord_list, db_coord_list)
len(q_coord_list), len(db_coord_list), len(q_crop_list), len(db_crop_list), panorama_id

(9, 8, 9, 8, 0)

In [52]:
q_crop_list[0].shape

(25, 82, 3)

In [3]:
# RootSIFT
# import the necessary packages

class RootSIFT:
	def __init__(self, max_kps):
		# initialize the SIFT feature extractor
		self.max_kps = max_kps
		self.extractor = cv2.SIFT_create(self.max_kps)
	def compute(self, image, kps, eps=1e-7):
		# compute SIFT descriptors
		kcps, descs = self.extractor.detectAndCompute(image,kps)
		# if there are no keypoints or descriptors, return an empty tuple
		if len(kps) == 0:
			return ([], None)
		# apply the Hellinger kernel by first L1-normalizing and taking the
		# square-root(L2-noramlized)
		descs /= (descs.sum(axis=1, keepdims=True) + eps)
		descs = np.sqrt(descs)
		#descs /= (np.linalg.norm(descs, axis=1, ord=2) + eps)
c		# return a tuple of the keypoints and descriptors
		return (kps, descs)

In [4]:
# RootSIFT
def rootsift_extractor(img):
    '''
    Description: extract \emph{sift} feature from given image
    Input: file_path - image path
    Output: des - a list of descriptors of all the keypoint from the image
    '''
    gray= cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rs = RootSIFT(99999) #full
    _, des = rs.compute(gray, None) 

    return des

In [ ]:
##### exec #####
# des = rootsift_extractor(q_crop_list[0])
# len(des)

In [22]:
#input : path_list

def ret_kp(q_path_list, db_path_list, panorama_id, dic_save_path, txt_save_path, matcher='bf', metric="l2", method='rootsift', \
           val = "union", cross_check=True, img_format='jpg'):
    
    '''
    panorama_id : ID of panorama, specific location identified number including query and database
    metric : str, how to compare with two key points, ["l1" : "NORM_L1", "l2" : "NORM_L2"]
    matcher : str, how to match key point with query and database, ['bf' : 'BF', 'flann' : 'FlannBased']
    method : str, how to extract feature from image, ['rootsift', 'sift', 'orb']
    val : str, how to calculate match score, ['union' : intersection / union, 'min' : intersection / min set]
    cross_check : bool, whether both description are all matched
    img_format : str, image format, [jpg, png, ...]
    '''
    
    #info
    print(f"{matcher}-{metric}-{method}")
    
    # check time
    start = time.perf_counter()
    
    # config
    matcher_dic = {'bf' : 'BF', 'flann' : 'FlannBased'}
    metric_dic = {"l1" : "NORM_L1", "l2" : "NORM_L2"}
    if matcher == 'bf':
        match = eval(f"cv2.BFMatcher(cv2.{metric_dic[metric]}, crossCheck={cross_check})") #cross_check 양쪽 des 매칭된 kp만 반영 ↔ 속도
    else: #flann
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        match = eval(f"cv2.FlannBasedMatcher({index_params}, {search_params})")
        
    # # time per query
    # time_list = []
    result_dic = {}
    
    for q_idx, q_path in enumerate(q_path_list):

        #time per query start
        q_t_b = time.perf_counter()

        # print(f"q : {q_idx+1}/{len(q_path_list)}") ####
        
        #extract feature
        try:
            des_q = eval(f"{method}_extractor('{q_path}')") 
        except:
            # print(q_path)
            print("extract q feature error")
    
        #q -> db match list
        match_list = []

        for db_idx, db_path in enumerate(db_path_list):

            # print(f"db img {idx_db}/{len(db_file_list)}")

            #extract feature
            #match btw query and db
            try:
                des_db = eval(f"{method}_extractor('{db_path}')")
                matches = match.match(des_q, des_db)
                
                #####eval#####
                # 매칭된 개수
                inter = len(matches)
                if val == 'union':
                    # 교집합/ 합집합    
                    union = (len(des_q)+len(des_db))
                    match_score = inter / union
                else :  # 'min'
                    # 교집합/ 작은 집합 수
                    min_set = min(len(des_q), len(des_db))
                    match_score = inter / min_set

                match_list.append((db_idx, match_score))
            except:
                # print(db_path)
                print("extract db feature error")

            #time per query check
            # print(f"search time:{time.perf_counter() - q_t_b}") ####
            # time_list.append(time.perf_counter() - q_t_b)

        #query db matching dic
        result_dic[q_idx] = match_list
    
    #check time
    print("ret_time", time.perf_counter()-start)
    
    #save result dict
    os.makedirs(dic_save_path + f"/{matcher}/{metric}/{method}/", exist_ok=True)
    with open(dic_save_path + f"/{matcher}/{metric}/{method}/dic_{panorama_id}" + ".p", 'wb') as fp: #rs : RootSIFT
        pickle.dump(result_dic, fp, protocol=pickle.HIGHEST_PROTOCOL)

    # save top1 pair .txt
    os.makedirs(txt_save_path + f"/{matcher}/{metric}/{method}/", exist_ok=True)
    out_file = txt_save_path + f"/{matcher}/{metric}/{method}/pair_{panorama_id}" + ".txt"
    f_out = open(out_file, 'w')

    for q_idx in list(result_dic.keys()):
        
        #query_name
        query_name = Path(q_path_list[q_idx]).stem
        
        # sort by match score(intersect/union)
        rank_list = sorted(result_dic[q_idx], key = lambda x: -x[1])

        for (db_idx, _)  in rank_list:
            
            #db_name
            db_name = Path(db_path_list[db_idx]).stem

            f_out.write(query_name + f'.{img_format},' + db_name + f'.{img_format}' + '\n')
            f_out.flush()
                    
    f_out.close()

##### exec ######
        
dic_save_path = "/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/result/result_dic/kp/"
txt_save_path = "/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/result/pair_txt/kp/"    

# check time
start = time.perf_counter()

for i, q in enumerate(pnorm_q_dic.keys()):
    
    #process
    print(f"ID : {i}/{len(pnorm_q_dic)}")
    
    for metric in ["l2"]:
        for matcher in ['bf']:
            for method in ['rootsift']:

                try: # query ID == db ID
                    # exec
                    ret_kp(q_path_list=pnorm_q_dic[q], db_path_list=pnorm_db_dic[q], panorama_id=q, dic_save_path=dic_save_path, \
                           txt_save_path=txt_save_path, matcher=matcher, metric=metric, method=method, img_format='jpg')

                except: # if query ID not in db ID
                    print("ID error")
                    print(f"query ID : {q}")
                    
#check time
print("total_time", time.perf_counter()-start)

# total_time 49.52079527702881

ID : 0/14
bf-l2-rootsift
ret_time 0.7395867279847153
ID : 1/14
bf-l2-rootsift
ret_time 0.031979760969989
ID : 2/14
bf-l2-rootsift
ret_time 0.4064304659841582
ID : 3/14
bf-l2-rootsift
ret_time 0.01321182498941198
ID : 4/14
bf-l2-rootsift
ret_time 0.14135883504059166
ID : 5/14
bf-l2-rootsift
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
extract db feature error
ret_time 

## utils
    - cp
    - rename

In [25]:
# cp pair.txt
# !cp /home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/result/pair_txt/kp/bf/l2/rootsift/*.txt /home/Signboard_Retrieval/roadview_384_crop/match_score/sift_best_pair

In [27]:
# rename pair.txt
# import shutil
# import os

# #rename folder
# rename_dir = '/home/Signboard_Retrieval/roadview_384_crop/match_score/sift_best_pair/'
# fname_list = os.listdir(rename_dir)

# for fname in fname_list:
#     new_fname = fname[:-4] + '_sift.txt'
#     shutil.move(f'{rename_dir}/{fname}', \
#                 f'{rename_dir}/{new_fname}')
#     print(fname, new_fname)

pair_170.txt pair_170_sift.txt
pair_10.txt pair_10_sift.txt
.ipynb_checkpoints .ipynb_checkpo_sift.txt
pair_200.txt pair_200_sift.txt
pair_100.txt pair_100_sift.txt
pair_150.txt pair_150_sift.txt
pair_0.txt pair_0_sift.txt
pair_110.txt pair_110_sift.txt
pair_140.txt pair_140_sift.txt
pair_120.txt pair_120_sift.txt
pair_20.txt pair_20_sift.txt
pair_160.txt pair_160_sift.txt
pair_180.txt pair_180_sift.txt
pair_130.txt pair_130_sift.txt
pair_190.txt pair_190_sift.txt


In [28]:
# input : path
def match_score(q_crop_path, db_crop_path):
    # Trace how to get the match score
    q_img = cv2.imread(q_crop_path) #query
    q_img_gray = cv2.cvtColor(q_img, cv2.COLOR_BGR2GRAY)
    q_img_flip0 = cv2.flip(q_img_gray, 0) #상하 반전
    q_img_flip1 = cv2.flip(q_img_gray, 1) #좌우 반전

    db_img = cv2.imread(db_crop_path)
    db_img_gray = cv2.cvtColor(db_img, cv2.COLOR_BGR2GRAY)
    
    # SIFT feature
    sift = cv2.SIFT_create()
    kp_q, des_q = sift.detectAndCompute(q_img_gray,None)
    kp_q0, des_q0 = sift.detectAndCompute(q_img_flip0,None)
    kp_q1, des_q1 = sift.detectAndCompute(q_img_flip1,None)
    kp_db, des_db = sift.detectAndCompute(db_img_gray,None)
    
    # KNN(K=2) Match including flipped img
    bf = cv2.BFMatcher(cv2.NORM_L2) #전수조사, Normalized L2

    match_num_list = []

    for des_q in [des_q, des_q0, des_q1]:
        matches = bf.knnMatch(des_q, des_db, k=2) #KNN-Matching(K=2) for match_score

        # ratio_threshold
        good_without_list = []

        cnt = 0
        for m,n in matches:
            cnt += 1
            if m.distance < (1.2/1.8)*n.distance: #2nd 최근접 이웃 거리의 특정 비율 보다 작은 1st 최근접 이웃만!

                good_without_list.append(m)

        #get match_num
        match_num_list.append(len(good_without_list))

    return max(match_num_list)

In [43]:
#input : path
def merge_topk(match_score_dir, q_crop_dir, db_crop_dir, panorama_id, topk, val=False):
    
    '''
    val: bool, whether evaluate result by mAP@topk or not
    '''
    
    
#     #config
#     os.system(f"chmod 777 {match_score_dir}/match_score")
#     os.system(f"export LD_LIBRARY_PATH={match_score_dir}/extLib/")
    
#     #match_score
#     method = ['sift', 'vit', 'ocr']
#     for m in method:
#         try:
#             os.system(f"""{match_score_dir}/match_score 
#                       {match_score_dir}/{m}_best_pair/pair_{panorama_id}_{method}.txt 
#                       {q_crop_dir}/{panorama_id}/ 
#                       {db_crop_dir}/{panorama_id}/ 
#                       {match_score_dir}/{m}_best_pair/pair_{panorama_id}_{m}.txt""")
#         except:
#             print(f"method {m} error")
    
    #merge
    method = ['sift', 'vit', 'ocr']
    merge_dict = {}
    
    for m in method:
        try:
            with open(f"{match_score_dir}/{m}_best_pair/pair_{panorama_id}_{m}.txt", "r") as f:
                for line in f.readlines():
                    data = line.strip().split(',')
                    img1 = data[0].split('.')[0]
                    img2 = data[1].split('.')[0]
                    score = match_score(q_crop_path=q_crop_dir+f"/{panorama_id}/{data[0]}",\
                                        db_crop_path=db_crop_dir+f"/{panorama_id}/{data[1]}")
                    if img1+'-'+img2 not in merge_dict: #3가지 방법에서 특정 q_db pair가 중복된다면, 점수를 합산!
                        merge_dict[img1+'-'+img2] = score
                    else:
                        merge_dict[img1+'-'+img2] += score
        except:
            print(f"method {m} error")
            
    #change form
    result_dict = {}
    
    for key in merge_dict:
        imgs = key.split('-')
        q = imgs[0]
        db = imgs[1]
        del imgs
        score = merge_dict[key]
        
        #특정 query에 대해, 모든 ref를 list에 모아주기
        if q not in result_dict: #이 query가 처음 나왔을 때
            result_dict[q] = [(db, int(score))]
        else: #query가 중복될 경우
            db_list = result_dict[q]
            db_list.append((db, int(score)))
            result_dict[q] = db_list
            
    #topk
    result_topk = {}
    
    for q in result_dict.keys():
    
        topk_list = sorted(result_dict[q], key=lambda x: -x[1])[:topk] #match_score descending
        topk_db = list(map(lambda x: x[0], topk_list)) #only db name
        result_topk[q] = topk_db
        
    #save result dict
    os.makedirs(save_path, exist_ok=True)
    with open(save_path + f"pair_{panorama_id}_top{topk}_final" + ".pkl", 'wb') as fp:
        pickle.dump(result_topk, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
    #eval mAP@topk
    #ex) query_name : '0_100_False'/ db_name : '2_100_False'
    if val:
        
        
        ap = 0
        for q_name in list(result_topk.keys()):
            #query_label
            q_lbl = int(q_name.split('_')[1])

            for rank, db_name in enumerate(result_topk[q_name]):
                #db_label
                db_lbl = int(db_name.split('_')[1])

                if (q_lbl == db_lbl) & (q_lbl != 100): #& ('True' not in q_name) : ##Count #100 not matched / + except change
                    ap += 1
        ap/=len([q for q in result_topk.keys() if ('100' not in q)]) #& ('True' not in q)]) ##Divide #except unmatched '100' / + except change
        print(f"AP@{topk} is {ap}") #result_topk 자체가 filter
        
    return result_topk, ap

# check time
start = time.perf_counter()

##### exec ######

match_score_dir = '/home/Signboard_Retrieval/roadview_384_crop/match_score/'
q_crop_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/q_crop_val/'
db_crop_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/db_crop_val/'
save_path = 'home/Signboard_Retrieval/roadview_384_crop/match_score/final_best_pair/'

topk = 1
ap_list = []
for i, q_id in enumerate(pnorm_q_dic.keys()):
    
    #process
    print(f"ID : {i}/{len(pnorm_q_dic)}")
    
    try: # panorama id가 있을 때
        result_topk, ap = merge_topk(match_score_dir=match_score_dir, q_crop_dir=q_crop_dir, db_crop_dir=db_crop_dir, \
                                 panorama_id=q_id, topk=topk, val=True)
        print(result_topk)
        ap_list.append(ap)
        
    except: # panorama id가 없을 때
        print(f"panorama_id {q} error")

print(f'mAP@{topk} : {np.mean(ap_list)}')

#check time
print("total_time", time.perf_counter()-start)

# total_time 57.5849611699814

# except changed
# total mAP@1 : 0.6119146005509642
# total mAP@5 : 0.8918732782369146

# include changed
# total mAP@1 : 0.6116179275270185
# total mAP@5 : 0.8817016317016317

ID : 0/14
method vit error
method ocr error
mAP@1 is 0.3333333333333333
{'0_100_False': ['2_100_False'], '1_100_False': ['0_100_False'], '2_100_False': ['1_100_False'], '3_100_False': ['0_100_False'], '4_100_False': ['8_6_False'], '5_1_False': ['0_100_False'], '6_100_False': ['0_100_False'], '7_2_False': ['4_2_False'], '8_3_False': ['6_3_False'], '9_4_False': ['7_5_False'], '10_5_False': ['8_6_False'], '11_100_False': ['0_100_False'], '12_6_False': ['1_100_False'], '13_100_False': ['0_100_False']}
ID : 1/14
method vit error
method ocr error
panorama_id 130 error
ID : 2/14
method vit error
method ocr error
mAP@1 is 0.16666666666666666
{'0_6_False': ['1_100_False'], '1_100_False': ['0_100_False'], '2_100_False': ['10_100_False'], '3_1_False': ['9_100_False'], '4_5_False': ['5_100_False'], '5_100_False': ['8_5_False'], '6_100_False': ['0_100_False'], '7_2_False': ['2_2_False'], '8_100_False': ['1_100_False'], '9_100_False': ['6_3_False'], '10_3_False': ['9_100_False'], '11_100_False': ['0

In [8]:
# Visualization

def txt2visualization(panorama_id, q_crop_dir, db_crop_dir, txt_dir, save_dir, topk):
    
    ID_list = [] #for all query identification
    txt_path = txt_dir + f"pair_{panorama_id}.txt" # load matching pair txt
    
    with open(txt_path, "r") as f:

        rows = topk # 몇 등까지 그릴 것인가?
        cols = 2

        for line in f.readlines():
            data = line.strip().split(',')
            query = data[0] # ex) 110.jpg or 111.png
            db = data[1]

            q_img_path = q_crop_dir + f'/{panorama_id}/{query}'
            db_img_path = db_crop_dir + f'/{panorama_id}/{db}'
            
            if query not in ID_list: #ID가 처음 나올 때

                #init
                #sketch
                fig = plt.figure(figsize=(7,10)) # rows*cols 행렬의 i번째 subplot 생성
                fig.patch.set_facecolor('xkcd:white') # 하얀 바탕으로
                
                rank = 0
                r_anchor = 0
                # sav_cnt = 0

            if rank >= topk:
                #for saving memory, close opened figure
                plt.cla()
                plt.clf()
                plt.close('all')
                pass
            
#                 if sav_cnt == 0: #not yet saved

#                     #save figure
#                     os.makedirs(Path(save_path).parent, exist_ok=True)
#                     plt.savefig(save_path, dpi = 100, bbox_inches='tight')

#                     #show figure - !show() 이후는 새로운 이미지(흰색 빈 이미지)가 저장됩니다.
#                     plt.tight_layout()
#                     plt.show()

#                     #count of save
#                     sav_cnt += 1

#                 else: #1번 저장하고 전부 pass
#                     pass

            else: ##visualization/ topk 결과 이미지만 생성

                if query not in ID_list: #ID가 처음 나올 때

                    #rank
                    rank += 1

                    #for identification
                    ID_list.append(query)

                    #ID query img 왼쪽 열에 그리기
                    ax = fig.add_subplot(rows, cols, 1)
                    q_name = Path(q_img_path).stem
                    query_img = cv2.imread(q_img_path)
                    ax.imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
                    ax.set_xticks([]), ax.set_yticks([])
                    ax.set_title(f"Q{q_name}", fontdict={'fontsize': 10})

                    #1st ID db img 오른쪽 열에 그리기
                    ax = fig.add_subplot(rows, cols, 2)
                    db_name = Path(db_img_path).stem
                    db_img = cv2.imread(db_img_path)
                    ax.imshow(cv2.cvtColor(db_img, cv2.COLOR_BGR2RGB))
                    ax.set_xticks([]), ax.set_yticks([])
                    ax.set_title(f"DB{db_name}_Rank: {rank}", fontdict={'fontsize': 10})

                else : #ID가 연달아 나올 때

                    #rank
                    rank += 1

                    # 1칸 아래로 내려서 그리자
                    r_anchor += 1

                    #2st ... db img 오른쪽 열에 그리기
                    ax = fig.add_subplot(rows, cols, 2+(cols*r_anchor))
                    db_name = Path(db_img_path).stem
                    db_img = cv2.imread(db_img_path)
                    ax.imshow(cv2.cvtColor(db_img, cv2.COLOR_BGR2RGB))
                    ax.set_xticks([]), ax.set_yticks([])
                    ax.set_title(f"DB{db_name}_Rank: {rank}", fontdict={'fontsize': 10})
                    
                #save figure
                save_path = save_dir + f'/{panorama_id}/top{topk}_{query}'
                os.makedirs(Path(save_path).parent, exist_ok=True)
                plt.savefig(save_path, dpi = 100, bbox_inches='tight')

#####exec ####
                
q_crop_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/q_crop/'
db_crop_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/db_crop/'
                
for i, q in enumerate(pnorm_q_dic.keys()):
    
    #process
    print(f"ID : {i}/{len(pnorm_q_dic)}")
    
    for metric in ["l2"]:
        for matcher in ['bf']:
            for method in ['rootsift']:
                
                # try: # query ID == db ID
                topk = 5
                txt_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/result/pair_txt/kp/' + f'{matcher}/{metric}/{method}/'
                save_dir = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/result/visualization/' + f'{matcher}/{metric}/{method}/'  

                txt2visualization(panorama_id=q, q_crop_dir=q_crop_dir, db_crop_dir=db_crop_dir, \
                                  txt_dir=txt_dir, save_dir=save_dir, topk=topk)
                
            v    # except: # if query ID not in db ID
                #     print("ID error")
                #     print(f"query ID : {q}")

ID : 0/97
ID : 1/97
ID : 2/97
ID : 3/97
ID : 4/97
ID : 5/97
ID : 6/97
ID : 7/97
ID : 8/97
ID : 9/97
ID : 10/97
ID : 11/97
ID : 12/97
ID : 13/97
ID : 14/97
ID : 15/97
ID : 16/97
ID : 17/97
ID : 18/97
ID : 19/97
ID : 20/97


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fee8aff7160> (for post_execute):


KeyboardInterrupt: 

# Input : Image(nd.array)

In [19]:
#### Path
###### panoramo db_root
pnorm_db_root = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/db_crop_val/'

pnorm_db_dic = {}
for db in os.listdir(pnorm_db_root):
    
    if db != 'labels':
        db_root = pnorm_db_root + db + '/'
        db_path_list = natsort.natsorted([db_root + db_name for db_name in os.listdir(db_root) if 'jpg' in db_name or 'png' in db_name])
        pnorm_db_dic[db] = db_path_list
        
pnorm_q_root = '/home/Signboard_Retrieval/roadview_384_crop/panorama_test/panorama_crop/q_crop_val/'

pnorm_q_dic = {}
for q in os.listdir(pnorm_q_root):
    
    if q != 'labels':
        q_root = pnorm_q_root + q + '/'
        q_path_list =natsort.natsorted( [q_root + q_name for q_name in os.listdir(q_root) if 'jpg' in q_name or 'png' in q_name])
        pnorm_q_dic[q] = q_path_list   

In [ ]:
# RootSIFT
# import the necessary packages

class RootSIFT:
	def __init__(self, max_kps):
		# initialize the SIFT feature extractor
		self.max_kps = max_kps
		self.extractor = cv2.SIFT_create(self.max_kps)
	def compute(self, image, kps, eps=1e-7):
		# compute SIFT descriptors
		kcps, descs = self.extractor.detectAndCompute(image,kps)
		# if there are no keypoints or descriptors, return an empty tuple
		if len(kps) == 0:
			return ([], None)
		# apply the Hellinger kernel by first L1-normalizing and taking the
		# square-root(L2-noramlized)
		descs /= (descs.sum(axis=1, keepdims=True) + eps)
		descs = np.sqrt(descs)
		#descs /= (np.linalg.norm(descs, axis=1, ord=2) + eps)
c		# return a tuple of the keypoints and descriptors
		return (kps, descs)

In [21]:
# RootSIFT
# input: file_path
def rootsift_extractor(file_path):
    '''
    Description: extract \emph{sift} feature from given image
    Input: file_path - image path
    Output: des - a list of descriptors of all the keypoint from the image
    '''
    img = cv2.imread(file_path)
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    rs = RootSIFT(99999) #full
    _, des = rs.compute(gray, None) 

    return des

In [17]:
# return q_crop_list, db_crop_list, panorama_id
# crop id는 임의로 부여하면 된다. why? 1쌍씩 들어오기 때문에.

def ret_kp(q_crop_list, db_crop_list, panorama_id, txt_save_path, matcher='bf', metric="l2", \
           val = "union", cross_check=True, img_format='jpg'):
    
    '''
    q_crop_list : list(np.array, ..., np.array), list of query panorama cropped images
    db_crop_list : list(np.array, ..., np.array), list of db panorama cropped images
    panorama_id : ID of panorama, specific location identified number including query and database
    txt_save_path : path of saving matching pair txt file
    ==
    metric : str, how to compare with two key points, ["l1" : "NORM_L1", "l2" : "NORM_L2"]
    matcher : str, how to match key point with query and database, ['bf' : 'BF', 'flann' : 'FlannBased']
    val : str, how to calculate match score, ['union' : intersection / union, 'min' : intersection / min set]
    cross_check : bool, whether both description are all matched
    img_format : str, image format, [jpg, png, ...]
    '''
    
    #info
    print(f"{matcher}-{metric}")
    
    # check time
    start = time.perf_counter()
    
    # config
    matcher_dic = {'bf' : 'BF', 'flann' : 'FlannBased'}
    metric_dic = {"l1" : "NORM_L1", "l2" : "NORM_L2"}
    if matcher == 'bf':
        match = eval(f"cv2.BFMatcher(cv2.{metric_dic[metric]}, crossCheck={cross_check})") #cross_check 양쪽 des 매칭된 kp만 반영 ↔ 속도
    else: #flann
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        match = eval(f"cv2.FlannBasedMatcher({index_params}, {search_params})")
        
    result_dic = {}
    
    for q_idx, q_image in enumerate(q_crop_list):
        
        #extract feature
        # try:
        des_q = rootsift_extractor(q_image) 
        # except:
        #     print("extract q feature error")
    
        #q -> db match list
        match_list = []

        for db_idx, db_image in enumerate(db_crop_list):

            #extract feature
            #match btw query and db
            try:
                des_db = rootsift_extractor(db_image) 
                matches = match.match(des_q, des_db)
                
                #####eval#####
                # 매칭된 개수
                inter = len(matches)
                if val == 'union':
                    # 교집합/ 합집합    
                    union = (len(des_q)+len(des_db))
                    match_score = inter / union
                else :  # 'min'
                    # 교집합/ 작은 집합 수
                    min_set = min(len(des_q), len(des_db))
                    match_score = inter / min_set

                match_list.append((db_idx, match_score))
            except:
                print("extract db feature error")

        #query db matching dic
        result_dic[q_idx] = match_list
    
    #ret time per panorama
    print("ret_time", time.perf_counter()-start)
    
    # save match pair .txt
    os.makedirs(txt_save_path, exist_ok=True)
    out_file = txt_save_path + f"/pair_{panorama_id}_sift" + ".txt"
    f_out = open(out_file, 'w')

    for q_idx in list(result_dic.keys()):
        
        # sort by match score(union)
        rank_list = sorted(result_dic[q_idx], key = lambda x: -x[1])
        
        for (db_idx, _)  in rank_list:    
            
            f_out.write(str(q_idx) + f'.{img_format},' + str(db_idx) + f'.{img_format}' + '\n')
            f_out.flush()
    f_out.close()

##### exec ######
txt_save_path = '/home/Signboard_Retrieval/roadview_384_crop/match_score/sift_best_pair/'

# check time
start = time.perf_counter()

# try:
# retrieval by keypoint matching
ret_kp(q_crop_list=q_crop_list, db_crop_list=db_crop_list, panorama_id=panorama_id, txt_save_path=txt_save_path)

# except: # if query ID not in db ID
#     print("ID error")
#     print(f"query ID : {q}")
                    
#check time
print("total_time", time.perf_counter()-start)

# total_time 49.52079527702881

bf-l2
ret_time 0.10497300798306242
total_time 0.10636653599794954


In [28]:
#input : image
def match_score(q_image, db_image):
    # Trace how to get the match score
    q_img_gray = cv2.cvtColor(q_image, cv2.COLOR_BGR2GRAY)
    q_img_flip0 = cv2.flip(q_img_gray, 0) #상하 반전
    q_img_flip1 = cv2.flip(q_img_gray, 1) #좌우 반전

    db_img_gray = cv2.cvtColor(db_image, cv2.COLOR_BGR2GRAY)
    
    # SIFT feature
    sift = cv2.SIFT_create()
    kp_q, des_q = sift.detectAndCompute(q_img_gray,None)
    kp_q0, des_q0 = sift.detectAndCompute(q_img_flip0,None)
    kp_q1, des_q1 = sift.detectAndCompute(q_img_flip1,None)
    kp_db, des_db = sift.detectAndCompute(db_img_gray,None)
    
    # KNN(K=2) Match including flipped img
    bf = cv2.BFMatcher(cv2.NORM_L2) #전수조사, Normalized L2

    match_num_list = []

    for des_q in [des_q, des_q0, des_q1]:
        matches = bf.knnMatch(des_q, des_db, k=2) #KNN-Matching(K=2) for match_score

        # ratio_threshold
        good_without_list = []

        cnt = 0
        for m,n in matches:
            cnt += 1
            if m.distance < (1.2/1.8)*n.distance: #2nd 최근접 이웃 거리의 특정 비율 보다 작은 1st 최근접 이웃만!

                good_without_list.append(m)

        #get match_num
        match_num_list.append(len(good_without_list))

    return max(match_num_list)

In [32]:
def merge_topk(match_score_dir, q_crop_list, db_crop_list, panorama_id, topk, val=False, save=False):
    
    '''
    val: bool, whether evaluate result by mAP@topk or not
    '''
    
#     #config
#     os.system(f"chmod 777 {match_score_dir}/match_score")
#     os.system(f"export LD_LIBRARY_PATH={match_score_dir}/extLib/")
    
#     #match_score
#     method = ['sift', 'vit', 'ocr']
#     for m in method:
#         try:
#             os.system(f"""{match_score_dir}/match_score 
#                       {match_score_dir}/{m}_best_pair/pair_{panorama_id}_{method}.txt 
#                       {q_crop_dir}/{panorama_id}/ 
#                       {db_crop_dir}/{panorama_id}/ 
#                       {match_score_dir}/{m}_best_pair/pair_{panorama_id}_{m}.txt""")
#         except:
#             print(f"method {m} error")
    
    #merge
    method = ['sift', 'vit', 'ocr']
    merge_dict = {}
    
    for m in method:
        try:
            with open(f"{match_score_dir}/{m}_best_pair/pair_{panorama_id}_{m}.txt", "r") as f:
                for line in f.readlines():
                    data = line.strip().split(',')
                    q_idx = data[0].split('.')[0]
                    db_idx = data[1].split('.')[0]
                    score = match_score(q_image=q_crop_list[int(q_idx)], db_image=db_crop_list[int(db_idx)])
                    if q_idx+'-'+db_idx not in merge_dict: #3가지 방법에서 특정 q_db pair가 중복된다면, 점수를 합산!
                        merge_dict[q_idx+'-'+db_idx] = score
                    else:
                        merge_dict[q_idx+'-'+db_idx] += score
        except:
            print(f"method {m} error")
            
    #change form
    result_dict = {}
    
    for key in merge_dict:
        imgs = key.split('-')
        q = imgs[0]
        db = imgs[1]
        del imgs
        score = merge_dict[key]
        
        #특정 query에 대해, 모든 ref를 list에 모아주기
        if q not in result_dict: #이 query가 처음 나왔을 때
            result_dict[q] = [(db, int(score))]
        else: #query가 중복될 경우
            db_list = result_dict[q]
            db_list.append((db, int(score)))
            result_dict[q] = db_list
            
    #topk
    result_topk = {}
    
    for q in result_dict.keys():
    
        topk_list = sorted(result_dict[q], key=lambda x: -x[1])[:topk] #match_score descending
        topk_db = list(map(lambda x: x[0], topk_list)) #only db name
        result_topk[q] = topk_db
    
    
    ##############################(optional) save& val###########
    
    #save result dict
    if save:
        os.makedirs(match_score_dir+'/final_best_pair/', exist_ok=True)
        with open(match_score_dir+'/final_best_pair/' + f"pair_{panorama_id}_top{topk}_final" + ".pkl", 'wb') as fp:
            pickle.dump(result_topk, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
    #eval mAP@topk
    #ex) query_name : '0_100_False'/ db_name : '2_100_False'
    if val:
        ap = 0
        for q_name in list(result_topk.keys()):
            #query_label
            q_lbl = int(q_name.split('_')[1])

            for rank, db_name in enumerate(result_topk[q_name]):
                #db_label
                db_lbl = int(db_name.split('_')[1])

                if (q_lbl == db_lbl) & (q_lbl != 100): #& ('True' not in q_name) : ##Count #100 not matched / + except change
                    ap += 1
        ap/=len([q for q in result_topk.keys() if ('100' not in q)]) #& ('True' not in q)]) ##Divide #except unmatched '100' / + except change
        print(f"AP@{topk} is {ap}") #result_topk 자체가 filter
        
        return result_topk, ap
    
    return result_topk

# check time
start = time.perf_counter()

##### exec ######

#맨 위에 선언해야 한다.
# match_score_dir = '/home/Signboard_Retrieval/roadview_384_crop/match_score/' 
# topk = 5

# return q_crop_list, db_crop_list, panorama_id
    
# try: # panorama id가 있을 때
result_topk = merge_topk(match_score_dir=match_score_dir, q_crop_list=q_crop_list, db_crop_list=db_crop_list, \
                             panorama_id=panorama_id, topk=topk, val=False, save=False)

print(result_topk)

# except: # panorama id가 없을 때
#     print(f"panorama_id {panorama_id} error")


# total_time 57.5849611699814

# except changed
# total mAP@1 : 0.6119146005509642
# total mAP@5 : 0.8918732782369146

# include changed
# total mAP@1 : 0.6116179275270185
# total mAP@5 : 0.8817016317016317

method vit error
method ocr error
{'0': ['4', '0', '2', '6', '1'], '1': ['6', '4', '1', '2', '7'], '2': ['0', '3', '6', '2', '5'], '3': ['6', '5', '3', '0', '4'], '4': ['6', '1', '0', '2', '3'], '5': ['1', '0', '5', '6', '4'], '6': ['1', '5', '2', '4', '6'], '7': ['6', '3', '0', '1', '2'], '8': ['7', '1', '4', '6', '2']}


In [48]:
# Visualization
# input image

def txt2visualization(panorama_id, q_crop_list, db_crop_list, txt_path, save_dir, topk):
    
    qid_list = [] #for all query identification
    
    with open(txt_path, "r") as f:

        rows = topk # 몇 등까지 그릴 것인가?
        cols = 2

        for line in f.readlines():
            data = line.strip().split(',')
            q_idx = int(data[0].split('.')[0]) # index
            db_idx = int(data[1].split('.')[0])
            
            if q_idx not in qid_list: #ID가 처음 나올 때

                #init
                #sketch
                fig = plt.figure(figsize=(7,10)) # rows*cols 행렬의 i번째 subplot 생성
                fig.patch.set_facecolor('xkcd:white') # 하얀 바탕으로
                
                rank = 0
                r_anchor = 0
                # sav_cnt = 0

            if rank+1 <= topk: #rank 0기준
                
                if q_idx not in qid_list: #ID가 처음 나올 때

                    #rank
                    rank += 1

                    #for identification
                    qid_list.append(q_idx)

                    #ID query img 왼쪽 열에 그리기
                    ax = fig.add_subplot(rows, cols, 1)
                    query_img = q_crop_list[q_idx]
                    ax.imshow(cv2.cvtColor(query_img, cv2.COLOR_BGR2RGB))
                    ax.set_xticks([]), ax.set_yticks([])
                    ax.set_title(f"Q{q_idx}", fontdict={'fontsize': 10})

                    #1st ID db img 오른쪽 열에 그리기
                    ax = fig.add_subplot(rows, cols, 2)
                    db_img = db_crop_list[db_idx]
                    ax.imshow(cv2.cvtColor(db_img, cv2.COLOR_BGR2RGB))
                    ax.set_xticks([]), ax.set_yticks([])
                    ax.set_title(f"DB{db_idx}_Rank: {rank}", fontdict={'fontsize': 10})

                else : #ID가 연달아 나올 때

                    #rank
                    rank += 1

                    # 1칸 아래로 내려서 그리자
                    r_anchor += 1

                    #2st ... db img 오른쪽 열에 그리기
                    ax = fig.add_subplot(rows, cols, 2+(cols*r_anchor))
                    db_img = db_crop_list[db_idx]
                    ax.imshow(cv2.cvtColor(db_img, cv2.COLOR_BGR2RGB))
                    ax.set_xticks([]), ax.set_yticks([])
                    ax.set_title(f"DB{db_idx}_Rank: {rank}", fontdict={'fontsize': 10})
                    
                #save figure
                save_path = save_dir + f'/{panorama_id}/top{topk}_{q_idx}.jpg'
                os.makedirs(Path(save_path).parent, exist_ok=True)
                plt.savefig(save_path, dpi = 100, bbox_inches='tight')

            else: ##visualization/ topk 결과 이미지만 생성
                
                #for saving memory, close opened figure
                plt.cla()
                plt.clf()
                plt.close('all')
                pass
            
#                 if sav_cnt == 0: #not yet saved

#                     #save figure
#                     os.makedirs(Path(save_path).parent, exist_ok=True)
#                     plt.savefig(save_path, dpi = 100, bbox_inches='tight')

#                     #show figure - !show() 이후는 새로운 이미지(흰색 빈 이미지)가 저장됩니다.
#                     plt.tight_layout()
#                     plt.show()

#                     #count of save
#                     sav_cnt += 1

#                 else: #1번 저장하고 전부 pass
#                     pass
    print('visualization done')
#####exec ####
                
txt_path='/home/Signboard_Retrieval/roadview_384_crop/match_score/sift_best_pair/pair_0_sift.txt'
save_dir = '/home/Signboard_Retrieval/roadview_384_crop/match_score/visualization/'
topk = 5

txt2visualization(panorama_id=panorama_id, q_crop_list=q_crop_list, db_crop_list=db_crop_list, \
                  txt_path=txt_path, save_dir=save_dir, topk=topk)

visualization done
